In [1]:
import numpy as np

In [67]:
class Net():
    
    def __init__(self):
        self.layers = []
        self.loss_type = LossType()
        
    def add(self, layer):
        self.layers.append(layer)
    
    def forward(self, A):
        for layer in self.layers:
            A = layer.forward(A)
        return A
    
    def backward_pass(self, dERROR, learning_rate, multiclass = False):
        if multiclass == True:
            #all layers without softmax
            layers = self.layers[0:-1]
        else:
            layers = self.layers
        
        for layer in reversed(layers):
            dERROR = layer.backward(dERROR, learning_rate)
            
    def get_accuracy(self, A, Y):
        preds = np.round(A, decimals=0)
        results = (Y == preds)
        results = np.squeeze(np.sum(results, axis = 1, keepdims=True))
        #numéro d'exemples
        m = A.shape[1]
        rate = results / m
        return rate
                
    def train(self, X, Y, learning_rate, epochs=20, multiclass = False):
        
        for e in range(epochs): 
            A = self.forward(X)
            dERROR = A - Y
            
            if multiclass == True:
                if e % 500 == 0: print("Epoch:", e, "Cost:", self.loss_type.multiclass(A, Y))
            else:
                if e % 500 == 0:
                    rate = self.get_accuracy(A,Y)
                    print("Epoch:", e, "Cost:", self.loss_type.binary(A, Y), "Acc:", rate)
                
            self.backward_pass(dERROR, learning_rate, multiclass=multiclass)

In [3]:
class LossType():
    
    def binary(self, A, Y):
        loss = - (np.log(A) * Y + np.log(1 - A) * (1 - Y))
        m = A.shape[1]
        cost = np.sum(loss, axis=1, keepdims=True) / m
        cost = np.round(np.squeeze(cost), 3)
        return cost
    
    def multiclass(self,Y,A):
        YL=np.multiply(A,Y)
        YL=YL[YL!=0]
        YL=-np.log(YL)
        YL=np.mean(YL)
        return YL

In [4]:
class LayerFC():
    
    def __init__(self, n_x, n_h):
        self.W = np.random.randn(n_h, n_x)
        self.b = np.zeros((n_h, 1))
    
    def forward(self, A_prev):
        self.A_prev = A_prev
        self.Z = np.dot(self.W, self.A_prev) + self.b
        return self.Z
    
    def backward(self, dERROR, learning_rate):
        m = dERROR.shape[1]
        dW = (1 / m) * np.dot(dERROR, self.A_prev.T)
        db = (1 / m) * np.sum(dERROR, axis=1, keepdims=True)
        dERROR = np.dot(self.W.T, dERROR)
        
        self.W = self.W - learning_rate * dW        
        self.b = self.b - learning_rate * db        
        
        return dERROR

In [5]:
class LayerSigmoid():
    
    def forward(self, Z):
        self.A = 1 / (1 + np.exp(-Z))
        return self.A
    
    def backward(self, dERROR, learning_rate):
        derivative = self.A * (1 - self.A)
        dERROR = dERROR * derivative        
        return dERROR


In [6]:
class LayerTanh():
    
    def forward(self, Z):
        self.A = np.tanh(Z)
        return self.A
    
    def backward(self, dERROR, learning_rate):
        derivative = 1 - np.square(self.A)
        dERROR = dERROR * derivative
        return dERROR
    

In [7]:
class LayerSoftmax():
    
    def forward(self, Z):
        self.A = np.exp(Z) / np.sum(np.exp(Z), axis=0)
        return self.A

In [8]:
def xor_data(n_x,m):
    X = np.random.randn(n_x, m)
    mask1 = X[0, :] > 0
    mask2 = X[1, :] > 0
    Y = np.logical_xor(mask1, mask2)
    Y = Y.reshape(1, m)
    
    return X,Y

In [9]:
X, Y = xor_data(2,300)

In [68]:
net = Net()
net.add(LayerFC(X.shape[0],2))
net.add(LayerTanh())
net.add(LayerFC(2, 2))
net.add(LayerTanh())
net.add(LayerFC(2, 1))
net.add(LayerSigmoid())

In [69]:
net.train(X, Y, epochs=5000, learning_rate=0.1, multiclass=False)

Epoch: 0 Cost: 0.738 Acc: 0.4866666666666667
Epoch: 500 Cost: 0.664 Acc: 0.4866666666666667
Epoch: 1000 Cost: 0.629 Acc: 0.5766666666666667
Epoch: 1500 Cost: 0.589 Acc: 0.7266666666666667
Epoch: 2000 Cost: 0.502 Acc: 0.8266666666666667
Epoch: 2500 Cost: 0.365 Acc: 0.91
Epoch: 3000 Cost: 0.277 Acc: 0.9533333333333334
Epoch: 3500 Cost: 0.231 Acc: 0.95
Epoch: 4000 Cost: 0.201 Acc: 0.9533333333333334
Epoch: 4500 Cost: 0.18 Acc: 0.9566666666666667


In [30]:
 def gen_3plasma_zones(sample_size, show_dist=False):
    
    cat_images = np.random.randn(sample_size, 2) + np.array([0, -3])
    mouse_images = np.random.randn(sample_size, 2) + np.array([3, 3])
    dog_images = np.random.randn(sample_size, 2) + np.array([-3, 3])

    X = np.vstack([cat_images, mouse_images, dog_images]).T 
    Y_temp = np.array([0]*sample_size + [1]*sample_size + [2]*sample_size)
    
    Y = np.zeros((3, sample_size * 3))
    for i in range(sample_size * 3):
        Y[Y_temp[i], i] = 1

    if show_dist:
        plt.scatter(X[0, :], X[1, :], c=Y_temp, cmap='plasma', s=100, alpha=0.6)  
        plt.show()
    
    return X, Y

X_train_multi, Y_train_multi = gen_3plasma_zones(300, False)

In [31]:
net = Net()
net.add(LayerFC(X_train_multi.shape[0],2))
net.add(LayerTanh())
net.add(LayerFC(2, 3))
net.add(LayerTanh())
net.add(LayerFC(3, 3))
net.add(LayerSoftmax())

In [32]:
net.train(X_train_multi, Y_train_multi, epochs=5000, learning_rate=0.09, multiclass=True)

Epoch: 0 Cost: 0.7829121790772259
Epoch: 500 Cost: 0.01502288982540576
Epoch: 1000 Cost: 0.007008511693580877
Epoch: 1500 Cost: 0.004556002219196872
Epoch: 2000 Cost: 0.003371629901704436
Epoch: 2500 Cost: 0.0026743124691669252
Epoch: 3000 Cost: 0.0022149268110831753
Epoch: 3500 Cost: 0.0018894778296183665
Epoch: 4000 Cost: 0.001646868743960777
Epoch: 4500 Cost: 0.0014590627895047102


In [239]:
pred = net.forward(X)
np.argmax(pred, axis = 0)

(2, 900)